本教程将介绍如何使用梯度提升（gradient boosting）来构建与优化模型。该方法在许多 Kaggle 竞赛中表现突出，并在多种数据集上达到领先水平（SOTA）。

# 引言

在本课程的大部分内容中，我们使用随机森林进行预测。随机森林通过对多棵决策树的预测取平均，往往优于单棵决策树。

我们把随机森林称作一种“集成方法”。按定义，**集成方法（ensemble methods）**会组合多个模型的预测（随机森林就是多棵树）。

接下来我们学习另一种集成方法：梯度提升。

# 梯度提升

**梯度提升**通过反复迭代的循环，不断向集成中加入新模型。

一开始，用一个较为朴素的模型初始化集成。（即便它很不准确，后续加入的模型会逐步纠正这些误差。）

然后进入循环：
- 使用当前集成对每条样本生成预测；预测时把集成中所有模型的预测相加；
- 用这些预测计算损失函数（例如[均方误差](https://en.wikipedia.org/wiki/Mean_squared_error)）；
- 利用损失函数来拟合将要加入集成的新模型，确定其参数，使其加入后能降低损失。（旁注：“梯度”指我们对损失函数执行[梯度下降](https://en.wikipedia.org/wiki/Gradient_descent)来确定新模型的参数。）
- 将新模型加入集成，然后……
- ……重复！

![tut6_boosting](https://storage.googleapis.com/kaggle-media/learn/images/MvCGENh.png)


# 示例

我们先把训练/验证数据加载到 `X_train`、`X_valid`、`y_train`、`y_valid`。

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

在这个示例中，我们将使用 XGBoost 库。**XGBoost** 是 **Extreme Gradient Boosting** 的缩写，是对梯度提升的高性能实现，提供了面向速度与性能的多种增强特性。（scikit-learn 也有自己的梯度提升实现，但 XGBoost 在一些技术点上具有优势。）

接下来我们导入 XGBoost 的 scikit-learn 接口（[`xgboost.XGBRegressor`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)）。这可以帮助我们像在 scikit-learn 中一样构建并拟合模型。你会看到 `XGBRegressor` 的许多可调参数，我们稍后介绍。

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

我们还会进行预测并评估模型。

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

# 参数调优

XGBoost 有一些关键参数会显著影响精度与训练速度。首先要理解的是：

### `n_estimators`
`n_estimators` 指的是上面建模循环迭代的次数，也等于集成中子模型（树）的数量。

- 过小会导致欠拟合（训练集与测试集上都不准）；
- 过大会导致过拟合（训练集上很准，但测试集上不准——而我们更关心测试集表现）。

典型取值范围在 100–1000 之间，但与下面的 `learning_rate` 强相关。

下面的代码展示了如何设定集成中的模型数量：

In [ ]:

my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

### `early_stopping_rounds`

`early_stopping_rounds` 提供了一种自动为 `n_estimators` 找到合适值的方式。早停会在验证分数不再提升时提前停止迭代，即使还未达到 `n_estimators` 的上限。实践中常用做法是把 `n_estimators` 设得偏大，再用 `early_stopping_rounds` 自动确定何时停止。

鉴于随机性，可能偶尔会有单轮分数未提升的现象，因此我们需要指定允许的连续不提升轮数。`early_stopping_rounds=5` 是个合理选择，即连续 5 轮分数未提升就停止训练。

使用早停时，需要预留一部分数据用于计算验证分数——通过 `eval_set` 参数指定。

**注意**：在较新版本的 XGBoost（1.6+）中，`early_stopping_rounds` 参数需要在 `XGBRegressor()` 构造函数中设置，而不是在 `fit()` 方法中。

我们可以在前面的示例上加入早停：

In [ ]:

my_model = XGBRegressor(n_estimators=500, early_stopping_rounds=5)
my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

如果随后你希望在“全部数据”上重新训练模型，可把 `n_estimators` 设置为早停过程中得到的最优值。

### `learning_rate`

与其简单地把每棵树的预测直接相加，不如先把每棵树的预测乘以一个较小的数（即**学习率**）再相加。

这意味着新加入的每棵树对整体的贡献更小，因此可以在不过拟合的前提下使用更大的 `n_estimators`。若启用了早停，合适的树数会被自动确定。

通常，更小的学习率 + 更多的树，会得到更准确的 XGBoost 模型；但也会带来更长的训练时间（循环迭代次数更多）。XGBoost 默认 `learning_rate=0.1`。

把上面的示例改为使用更小学习率的代码如下：

In [ ]:

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, early_stopping_rounds=5)
my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

### `n_jobs`
在更大的数据集上，训练耗时更敏感，此时可以利用并行来加速建模。通常把 `n_jobs` 设为机器 CPU 的核心数。对小数据集则意义不大。

需要注意的是，这不会让模型本身更好；为了“缩短拟合时间”而做的微优化通常意义有限。不过在大数据集上，确实能显著缩短 `fit` 的等待时间。

修改示例如下：

In [ ]:

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4, early_stopping_rounds=5)
my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

# 结论

[XGBoost](https://xgboost.readthedocs.io/en/latest/) 是处理标准表格数据（比如 Pandas DataFrame）的一线库（与图像、视频等“特殊数据”不同）。通过细致的参数调优，你可以训练出非常准确的模型。

# 轮到你了

在 **[下一道练习](https://www.kaggle.com/kernels/fork/3370271)** 中用 XGBoost 训练你的模型吧！

---




*有问题或想交流？欢迎访问[课程讨论区](https://www.kaggle.com/learn/intermediate-machine-learning/discussion)与其他学习者交流。*